In [98]:
import spotipy
import spotipy.util as util
from urllib.request import urlretrieve
import pandas as pd
import numpy as np
import json
import dateutil.parser as dp

In [123]:
def get_token(username,scope):
    # how to not leave them inside source code??
    client_id='491a6bacf0324a5f9fae53586ed2d985'
    client_secret='74f7c2bc50c54d86b8ba69e25ec638c5'
    redirect_url='http://localhost/'
    token=util.prompt_for_user_token(username,
                               scope,
                               client_id=client_id,
                               client_secret=client_secret,
                               redirect_uri=redirect_url)
    return token

In [126]:
username = 'kaan3'
scope = 'user-top-read'
token = get_token(username,scope)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=491a6bacf0324a5f9fae53586ed2d985&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%2F&scope=user-top-read in your browser


Enter the URL you were redirected to: http://localhost/?code=AQBMxcVLA6sUJWQnuHYIkPDyY-borxX2wDRlLDZgfCl65DeMN_O9d2iwR8EbRARY3Yf2cFg4j6i_y9UUzPB_PvLET4dULBCvMESaapNfXwQX5a6Shymk5ydjXFfgOtTmmT6ftSqG_Ex8IKWFEKLIzI1BYA7x-ZjGr0MZGHE9zkNIEWUqf44-5t1-lYK-klmAojdbbAFFNNc




In [127]:
sp = spotipy.Spotify(auth=token)

In [4]:
def get_playlist_ids(username):
    ids=[]
    names=[]
    playlists = sp.user_playlists(username)
    for item in playlists['items']:
        names.append(item.get('name'))
        ids.append(item.get('id'))
    return ids,names

In [5]:
ids,names = get_playlist_ids(username)
print(ids)
print(names)

['1zq54v3vEEsho37RJZ5yTL', '5qZnUPIK06s2RbNbenB162', '2gE9DHaBDLJcUrLjuw3l4R', '5iLE4ZAb3RSUfUYvYAQnDf', '6yNmu3t206MIH38EnKh9lG', '1xQoOU4WNtz1TRY6vLHGUC', '5ABcvX6KAzbJhLu8gqNKat', '0KMhWZ7V3R5mTsulpFZvEv', '7eL7JGXezKKOyyOHcxRYCw']
['❄️💊', 'Puis danse', 'Dalgalarla Kabaran Konforlu Yastıklar', 'ｌｉｓｔｅｎ\u3000ｈｅｒｅ\u3000ｕ\u3000ｌｉｌ\u3000ｓｈｉｔ', 'RIP Vinyl aka NO SKIP', 'EZ  👂🏼', 'Kukumba', 'gnj', '-']


In [136]:
a=sp.current_user_top_artists()
for i in a['items']:
    print(i.get('name'),i.get('id'))

The Asteroids Galaxy Tour 68g1s6VqLLLBI3tXR0Bb7C
Anderson .Paak 3jK9MiCrA42lLAdMGUZpwa
Vendredi sur Mer 0wuuYZFptujAsRthrdea2B
SAINt JHN 0H39MdGGX6dbnnQPt6NQkZ
Vulfpeck 7pXu47GoqSYRajmBCjxdD6
Khruangbin 2mVVjNmdjXZZDvhgQWiakk
Ratatat 57dN52uHvrHOxijzpIgu3E
Saib 6N4HlHINMvoTyAL0yhBUCk
PillowTalk 0c8mm66bWaIlx5jdUhyYne
Led Zeppelin 36QJpDe2go2KgaRleHCDTp
Nilüfer Yanya 09kXLeOXRyfNQMXRaDO4qA
KAYTRANADA 6qgnBH6iDM91ipVXv28OMu
L'Impératrice 4PwlsrN0t5mLN0C827cbEU
Lewis OfMan 1hkRfKGoJisJDbo6eSf1pg
Shannon and The Clams 6A5Ns1SpGWTt8SzXPwiqVE
The Growlers 2RhgnQNC74QoBlaUvT4MEe
Menahan Street Band 1PryMSya1JnSAlcwYawCxp
Corine 6vs7gfG3OVDyBiY7loLsyQ
Flamingosis 75cW8FFekyCjj0mfZM1Gfb
Toto 0PFtn5NtBbbUNbU9EAmIWF


In [157]:
class User:

    def __init__(self,username):
        self.username = username
        self.get_user_info()
        self.playlists = self.get_playlists()
        self.top_tracks = self.get_top_list(tolist='tracks')
        self.top_artists = self.get_top_list(tolist='artists')


    '''
    Methods to get user info with spotipy module 
    and parse to class attributes
    '''
    def get_user_info(self):
        user = sp.me()
        self.display_name = user['display_name']
        self.profimg_irl = user['images'][0]['url']

    def get_playlists(self):
        pl_dict = {}
        try:
            playlists = sp.user_playlists(self.username)
        except:
            scope = 'user-library-read'
            token = get_token(self.username,scope)
            sp=spotipy.Spotify(auth=token)
            playlists = sp.user_playlists(self.username)
            
        for item in playlists['items']:
            pl_dict[item.get('name')] = Playlist(self.username,item.get('id'))
        return pl_dict

    def get_top_list(self,limit=10,time_range='medium_term',tolist='tracks'):
        top_list ={}
        try:
            _ = sp.current_user_top_tracks(limit=limit,time_range=time_range)
        except:
            scope = 'user-top-read'
            token = get_token(self.username,scope)
            sp=spotipy.Spotify(auth=token)
            _ = sp.current_user_top_tracks(limit=limit,time_range=time_range)
        

        if tolist=='tracks':
            tops=sp.current_user_top_tracks(limit=limit,time_range=time_range)
            for i,top in enumerate(tops['items']):
                top_list[str(i+1)]=Track(top.get('id'))
        elif tolist=='artists':
            tops=sp.current_user_top_artists(limit=limit,time_range=time_range)
            for i,top in enumerate(tops['items']):
                top_list[str(i+1)]=Artist(top.get('id'),artistname=top.get('name'))
        else:
            pass
        return top_list


    #TO DO:
    #user music taste as an attribute extracted from playlist data
       #can be top 5 genres
    # user recently played
    # user top artists

class Playlist:
    
    def __init__(self,username,plid):
        self.username = username
        self.plid = plid
        self.get_pl_info()
    '''
    Methods to get track info with spotipy module 
    and parse to class attributes
    '''
    def get_pl_info(self):
        p=sp.user_playlist(self.username,self.plid)
        self.name = p['name']
        self.isPublic = p['public']
        self.isCollab = p['collaborative']
        self.trackCount = p['tracks']['total']
    
    def read_tracks(self):
        p=sp.user_playlist(self.username,self.plid)
        for t in p['tracks']['items']:
            # convert date string to datetime object
            # read the tracks to somewhere
            pass
#             print(t['track']['id'],t['added_at'])
    '''
    Visualization methods
    '''
    def extract_features(self):
        pass
        # music taste change through time(what is taste here?)

class Track:
    def __init__(self, trackid, **kwargs):
        self.trackid = trackid
        self.date_added = kwargs.get('date_added','')
        self.get_track_info()
    
    '''
    Methods to get track info with spotipy module 
    and parse to class attributes
    '''
    def get_track_info(self):
        t=sp.track(self.trackid)
        self.name = t['name']
        self.artist = [x['name'] for x in t['artists']]
        self.artist_id = [x['id'] for x in t['artists']]
        self.album = t['album']['name']
        self.duration = t['duration_ms']
        self.popularity = t['popularity']
        self.preview_url = t['preview_url']
        self.audio_features=sp.audio_features(self.trackid)[0]
        self.track_genre=self.get_genre(self.artist_id)
        
    
    def get_genre(self,artist_id):
        genre=[]
        if not type(artist_id) is list:
            artist_id=[artist_id]
        for i in artist_id:
            a=sp.artist(i)
            genre += a['genres']
        return list(set(genre))

    '''
    Display Methods
    '''
    def display_track_info(self):
        df_info = pd.DataFrame(data=[vars(self)],index=[0])
        pd.set_option('display.max_rows', len(df_info))
        print(df_info)
    
    def display_track_features(self):
        df_feature = pd.DataFrame(data=self.audio_features,index=[0])
        pd.set_option('display.max_rows', len(df_feature))
        print(df_feature)

class Artist:
        def __init__(self, artistid,**kwargs):
            self.artistid = artistid
            self.atistname = kwargs.get('artistname','')
            self.top_tracks = self.get_top_tracks()

        def get_top_tracks(self):
            pass

        def display_artist(self):
            print(self.artistname)



In [142]:
tmp=Playlist(username,'7eL7JGXezKKOyyOHcxRYCw')
tmp.read_tracks()

In [27]:
tmp=sp.user_playlist(username,'7eL7JGXezKKOyyOHcxRYCw')

print(tmp['tracks']['items'][0]['track']['id'])
print(json.dumps(tmp['tracks']['items'],indent=2))

01TeWe9FtQOOoO1xD8idoU
[
  {
    "added_at": "2019-01-29T10:11:01Z",
    "added_by": {
      "external_urls": {
        "spotify": "https://open.spotify.com/user/kaan3"
      },
      "href": "https://api.spotify.com/v1/users/kaan3",
      "id": "kaan3",
      "type": "user",
      "uri": "spotify:user:kaan3"
    },
    "is_local": false,
    "primary_color": null,
    "track": {
      "album": {
        "album_type": "album",
        "artists": [
          {
            "external_urls": {
              "spotify": "https://open.spotify.com/artist/75cW8FFekyCjj0mfZM1Gfb"
            },
            "href": "https://api.spotify.com/v1/artists/75cW8FFekyCjj0mfZM1Gfb",
            "id": "75cW8FFekyCjj0mfZM1Gfb",
            "name": "Flamingosis",
            "type": "artist",
            "uri": "spotify:artist:75cW8FFekyCjj0mfZM1Gfb"
          }
        ],
        "available_markets": [
          "AD",
          "AE",
          "AR",
          "AT",
          "AU",
          "BE",
         

In [158]:
u=User('kaan3')



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=491a6bacf0324a5f9fae53586ed2d985&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%2F&scope=user-library-read in your browser


Enter the URL you were redirected to: http://localhost/?code=AQBzIroV_C-iBwxvda1FmLT-nmg-B8IH7gD51lWehMeSoPzvfxV1ZZ44eq6xRamGme13b21vz1dKJQezLaN3t3eWZxwFGYVNIzRy3FDLXknZpJJCUrATiqJ7XIgyZniX1loJdbV6HkN-zFKPa_Ahh8rd3q4R_2mFKAd5HSxqMSkDemTYYOjfP8dCl3ujnlMiOvWU5KpF5ztljFIk




            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
   

In [159]:
vars(u)

{'username': 'kaan3',
 'display_name': 'Kaan Çetindağ',
 'profimg_irl': 'https://profile-images.scdn.co/images/userprofile/default/3bf62d1d1a4ae1db02498a2c518951ad538631a5',
 'playlists': {'❄️💊': <__main__.Playlist at 0x120f4ee10>,
  'Puis danse': <__main__.Playlist at 0x11e92d7d0>,
  'Dalgalarla Kabaran Konforlu Yastıklar': <__main__.Playlist at 0x120f43e10>,
  'ｌｉｓｔｅｎ\u3000ｈｅｒｅ\u3000ｕ\u3000ｌｉｌ\u3000ｓｈｉｔ': <__main__.Playlist at 0x11fe78ed0>,
  'RIP Vinyl aka NO SKIP': <__main__.Playlist at 0x11fe78510>,
  'EZ  👂🏼': <__main__.Playlist at 0x11fe78bd0>,
  'Kukumba': <__main__.Playlist at 0x11e92d6d0>,
  'gnj': <__main__.Playlist at 0x11e92d890>,
  '-': <__main__.Playlist at 0x11e92da50>},
 'top_tracks': {'1': <__main__.Track at 0x11e92d750>,
  '2': <__main__.Track at 0x120e0cc90>,
  '3': <__main__.Track at 0x120ecf890>,
  '4': <__main__.Track at 0x120ec4fd0>,
  '5': <__main__.Track at 0x120e0cf10>,
  '6': <__main__.Track at 0x120ee9d50>,
  '7': <__main__.Track at 0x120f4e690>,
  '8': <__

In [70]:
t1.display_track_info()

                  trackid date_added       name                   artist  \
0  01TeWe9FtQOOoO1xD8idoU        ttt  Groovin '  [Flamingosis, Yung Bae]   

                                          artist_id        album  duration  \
0  [75cW8FFekyCjj0mfZM1Gfb, 30FDJPN3RtwJZ20g5YGCRX]  Kahunastyle    213559   

   popularity                                        preview_url  \
0          43  https://p.scdn.co/mp3-preview/e28b57ceee500fa4...   

                                      audio_features  \
0  {'danceability': 0.707, 'energy': 0.886, 'key'...   

                          track_genre  
0  [vaporwave, chillhop, future funk]  


In [67]:
a={'a':[1,2,3],'b':[6,7],'c':'f'}
print(a.values())
f=pd.DataFrame(data=[a])
f

dict_values([[1, 2, 3], [6, 7], 'f'])


,a,b,c
0,"[1, 2, 3]","[6, 7]",f


In [60]:
a=sp.artist('30FDJPN3RtwJZ20g5YGCRX')
a['genres']

['future funk', 'vaporwave']

In [106]:
f=sp.audio_features('01TeWe9FtQOOoO1xD8idoU')[0]
print(f)
def my_func(key=None):
    print(key)
   #do the real stuff

my_func(**f)


{'danceability': 0.707, 'energy': 0.886, 'key': 11, 'loudness': -3.432, 'mode': 1, 'speechiness': 0.0603, 'acousticness': 0.000699, 'instrumentalness': 0.396, 'liveness': 0.117, 'valence': 0.75, 'tempo': 117.976, 'type': 'audio_features', 'id': '01TeWe9FtQOOoO1xD8idoU', 'uri': 'spotify:track:01TeWe9FtQOOoO1xD8idoU', 'track_href': 'https://api.spotify.com/v1/tracks/01TeWe9FtQOOoO1xD8idoU', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/01TeWe9FtQOOoO1xD8idoU', 'duration_ms': 213559, 'time_signature': 4}


TypeError: my_func() got an unexpected keyword argument 'danceability'

In [85]:
t=sp.track('01TeWe9FtQOOoO1xD8idoU')
print(json.dumps(t,indent=2))

{
  "album": {
    "album_type": "album",
    "artists": [
      {
        "external_urls": {
          "spotify": "https://open.spotify.com/artist/75cW8FFekyCjj0mfZM1Gfb"
        },
        "href": "https://api.spotify.com/v1/artists/75cW8FFekyCjj0mfZM1Gfb",
        "id": "75cW8FFekyCjj0mfZM1Gfb",
        "name": "Flamingosis",
        "type": "artist",
        "uri": "spotify:artist:75cW8FFekyCjj0mfZM1Gfb"
      }
    ],
    "available_markets": [
      "AD",
      "AE",
      "AR",
      "AT",
      "AU",
      "BE",
      "BG",
      "BH",
      "BO",
      "BR",
      "CA",
      "CH",
      "CL",
      "CO",
      "CR",
      "CY",
      "CZ",
      "DE",
      "DK",
      "DO",
      "DZ",
      "EC",
      "EE",
      "EG",
      "ES",
      "FI",
      "FR",
      "GB",
      "GR",
      "GT",
      "HK",
      "HN",
      "HU",
      "ID",
      "IE",
      "IL",
      "IN",
      "IS",
      "IT",
      "JO",
      "JP",
      "KW",
      "LB",
      "LI",
      "LT",
      

In [21]:
def get_playlist_ids(username):
    ids=[]
    names=[]
    playlists = sp.user_playlists(username)
    for item in playlists['items']:
        names.append(item.get('name'))
        ids.append(item.get('id'))
    return ids,names

def retrieve_tracks(username,playlist_id,fields)
    tracks=[]
    temp_track=[]
    pl_json=sp.user_playlist(username,playlist_id,fields)
    

def get_track_details(username, playlist):
    sp.user_playlist(fields=)
    pass

def display_playlist(playlist_id):
    pass

def display_track(track_id):
    pass

In [74]:
print(names[-1])
p=sp.user_playlist(username,ids[-1])
print(p['name'])
print(type(p))
print(json.dumps(p,indent=2))

-
-
<class 'dict'>
{
  "collaborative": false,
  "description": "",
  "external_urls": {
    "spotify": "https://open.spotify.com/playlist/7eL7JGXezKKOyyOHcxRYCw"
  },
  "followers": {
    "href": null,
    "total": 0
  },
  "href": "https://api.spotify.com/v1/playlists/7eL7JGXezKKOyyOHcxRYCw",
  "id": "7eL7JGXezKKOyyOHcxRYCw",
  "images": [
    {
      "height": 640,
      "url": "https://i.scdn.co/image/929c849625e8d974034591427b6bc6b61da12463",
      "width": 640
    }
  ],
  "name": "-",
  "owner": {
    "display_name": "Kaan \u00c7etinda\u011f",
    "external_urls": {
      "spotify": "https://open.spotify.com/user/kaan3"
    },
    "href": "https://api.spotify.com/v1/users/kaan3",
    "id": "kaan3",
    "type": "user",
    "uri": "spotify:user:kaan3"
  },
  "primary_color": null,
  "public": true,
  "snapshot_id": "NCw5MjVkNzZiN2RkMDkyZjQ1N2RhZWE4MDMzMzgwNGQ0MTZmYTVjOWM0",
  "tracks": {
    "href": "https://api.spotify.com/v1/playlists/7eL7JGXezKKOyyOHcxRYCw/tracks?offset=0&limit

In [15]:
print(json.dumps(playlists,indent=2))

{
  "href": "https://api.spotify.com/v1/users/kaan3/playlists?offset=0&limit=50",
  "items": [
    {
      "collaborative": false,
      "external_urls": {
        "spotify": "https://open.spotify.com/playlist/1zq54v3vEEsho37RJZ5yTL"
      },
      "href": "https://api.spotify.com/v1/playlists/1zq54v3vEEsho37RJZ5yTL",
      "id": "1zq54v3vEEsho37RJZ5yTL",
      "images": [
        {
          "height": null,
          "url": "https://pl.scdn.co/images/pl/default/a58ab20825533c353b8f2c6078487ed13780bd17",
          "width": null
        }
      ],
      "name": "\u2744\ufe0f\ud83d\udc8a",
      "owner": {
        "display_name": "Kaan \u00c7etinda\u011f",
        "external_urls": {
          "spotify": "https://open.spotify.com/user/kaan3"
        },
        "href": "https://api.spotify.com/v1/users/kaan3",
        "id": "kaan3",
        "type": "user",
        "uri": "spotify:user:kaan3"
      },
      "primary_color": null,
      "public": true,
      "snapshot_id": "MTc1LDEyMTkyYjBjN